# Acceso a Pfam

Buscamos datos de todas la familias de Pfam

La url base de Pfam es "https://pfam.xfam.org/", pero en algunos sistemas no se
puede acceder bien, en esos casos, se puede usar "http://pfam.xfam.org/" (notar que es "https" en un
caso y "http" en el otro.

Pfam provee unos pocos servicios que se pueden usar para acceder programaticamente.

In [ ]:
library(httr)
library(xml2)

## Recuperar datos de las familias disponibles

Uno de los servicios permite recuperar el nombre, código de acceso y descripción de todas las familias de proteínas disponibles.

In [ ]:
url   <- "http://pfam.xfam.org/families"
respuesta <- GET(url, query=list(output="text"))

In [ ]:
families <-read.table(
    text=content(respuesta),
    header=FALSE,
    sep="\t",
    quote="",
    comment.char="#",
    col.names=c('Accession', 'Name', 'Description')
)
head(families)

## Conversión de identificadores

Podemos convertit el código de acceso y el identificador de nombres.


In [ ]:
# url <- "http://pfam.xfam.org/family/id?output=xml&acc=PF00082"
url <- "http://pfam.xfam.org/family/id"

family_acc <- 'PF00082'
respuesta <- GET(url, query=list(output="xml", acc=family_acc))
data <- content(respuesta)

In [ ]:
xml_attrs(xml_children(data))

## Recuperar anotaciones

In [ ]:
url <- "http://pfam.xfam.org/family"

family_acc <- 'PF00082'
respuesta <- GET(url, query=list(output="xml", acc=family_acc))
data <- as_list(content(respuesta))

In [ ]:
names(data$pfam$entry)

In [ ]:
names(data$pfam$entry$curation_details)

Recuperamos el tipo de familia, la descripción, la cobertura promedio, y la longitud promedio.

In [ ]:
# descripcion
trimws(data$pfam$entry$description)

# tipo
unlist(data$pfam$entry$curation_details$type)

# cobertura promedio
unlist(data$pfam$entry$curation_details$av_coverage)

# cobertura promedio
unlist(data$pfam$entry$curation_details$av_length)

## Buscar datos de una proteína

Podemos buscar si una proteína 

In [ ]:
url <- "http://pfam.xfam.org/protein"

uniprot <- 'P0AA25'
respuesta <- GET(url, query=list(output="xml", entry=uniprot))
data <- content(respuesta)
data_list <- as_list(data)

In [ ]:
data_list

In [ ]:
# Recupero la secuencia en formato fasta
data_list$pfam$entry$sequence

In [ ]:
# Recupero datos del match de la proteína los pfams
matches <- xml_find_all(data, ".//d1:match")

# Datos del match
xml_attrs(matches[[1]])

# Datos de la unibación del match
xml_attrs(xml_children(matches[[1]]))



In [ ]:
## Buscar dominios en un secuencia de una proteína
url <- "https://pfam.xfam.org/search/sequence"
seq <- "NQEFLSQEFPDIHILKIDVDECEELTVDYEIQDMPTIKKKKTLESFTGANIEQVKHYS"

body <- list(
    seq = seq,
    output="xml"
)

response <- POST(
    url,
    body=body
)

In [ ]:
data <- as_list(content(response))
data$html$body$jobs$job$result_url

In [ ]:
url <- paste0("https://pfam.xfam.org/", unlist(data$html$body$jobs$job$result_url))
response_job <- GET(url, query=list(output="xml"))

In [ ]:
cat(content(response_job, as="text"))

In [ ]:
data <- content(response_job)

matches <- xml_find_all(data, './/match')

xml_attrs(matches)

In [ ]:
location <- xml_child(matches, "location")
xml_attrs(location)

## Descargar alineamientos

Podemos descargas varios tipos de alineamientos de pfam


In [ ]:
# Descargamos alineamientos Seed
accession <- "PF00085"
url <- paste0("https://pfam.xfam.org/family/", accession, "/alignment/seed")
seed_msa <- GET(url)

In [ ]:
# Lo guardamos en el disco
write(content(seed_msa, as="text"), file="pf_00085_seed.stockholm")

In [ ]:
# Descargamos alineamientos full

accession <- "PF00085"
url <- paste0("https://pfam.xfam.org/family/", accession, "/alignment/full")
full_msa <- GET(url)

In [ ]:
# Lo guardamos en el disco
write(content(full_msa, as="text"), file="pf_00085_full.stockholm")

Podemos descargar  proteomas representativos con diferente grado de indentidad de secuencia,
que pertenecen a una familia.

- RP15 Proteinas con 15% de identidad o mayor
- RP35 Proteinas con 35% de identidad o mayor
- RP55 Proteinas con 55% de identidad o mayor
- RP75 Proteinas con 75% de identidad o mayor

Para ello, hay que modificar la url.

In [ ]:
url <- paste0("https://pfam.xfam.org/family/", accession, "/alignment/rp15")
url <- paste0("https://pfam.xfam.org/family/", accession, "/alignment/rp35")
url <- paste0("https://pfam.xfam.org/family/", accession, "/alignment/rp55")
url <- paste0("https://pfam.xfam.org/family/", accession, "/alignment/rp75")

# Acceso a RSCB PDB

Al igual que NCBI y Uniprot, PDB permite realizar búsqueda de texto.
Aunque estas son bastante más complicadas de realizar que en los servicios anteriores.

La consulta se tiene que escribir en formato **json**.

Vamos a provar con una búsquea simple.

La respuesta también está en formato json.

In [ ]:
library(httr)

In [ ]:
url <- "https://search.rcsb.org/rcsbsearch/v1/query"

query <- '{
  "request_options": {
     "return_all_hits": true
  },
  "query": {
    "type": "terminal",
    "service": "text",
    "parameters": {
      "value": "phosphofructokinase"
    }
  },
  "return_type": "entry"
}'

respuesta <- GET(url, query=list(json=query))

In [ ]:
pdb_search <- content(respuesta)

In [ ]:
# Recuperamos la cantidad de hits.
pdb_search$total_count

In [ ]:
# Recuperamos todos los resultados
length(pdb_search$result_set)

In [ ]:
pdb_ids <- sapply(pdb_search$result_set, function(x) x$identifier)

In [ ]:
install.packages("Rpdb")
library(bio3d)
library(Rpdb)

In [ ]:
# Podemos descargar el archivo PDB

bio3d::get.pdb(pdb_ids[1])

In [ ]:
# Podemos leer los datos del PDB
pdb1 <- bio3d::read.pdb('./1MTO.pdb')

In [ ]:
install.packages("remotes")
remotes::install_github("nvelden/NGLVieweR")

In [ ]:
library(NGLVieweR)

In [ ]:
NGLVieweR("./1MTO.pdb") %>% addRepresentation("cartoon")